# Comparison of Boschloo, Barnard and fisher exact test

In statistics, There are three main tests used in the analysis of 2×2 contingency tables with one margin fixed. They are named after the one who discovered it: R. D. Boschloo, [Barnard](https://en.wikipedia.org/wiki/George_Alfred_Barnard), and the very well known [Ronald Fisher](https://en.wikipedia.org/wiki/Ronald_Fisher)

The first one in historical timeline is Fisher exact test. 

The purpose of this Notebook is to discover which one is the most powerful and which condition we should use one over another.

In [ ]:
from dataclasses import make_dataclass, field
from functools import partial, reduce

import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import boschloo_exact, fisher_exact, barnard_exact

In [ ]:
FUNCTIONS = [boschloo_exact, fisher_exact, barnard_exact]

In [ ]:
def get_binom_samples(n, p, size=1000):
    tables = stats.binom.rvs(n, p, size=size)
    return tables

ResPvalues = make_dataclass('ResPvalues',
                   [('boschloo_exact', float, field(default=0)),
                    ('barnard_exact', float, field(default=0)),
                    ('fisher_exact', float, field(default=0)),],)

def get_pvalues(table, alt="two-sided"):
    res = ResPvalues()
    for fn in FUNCTIONS:
        if fn.__name__ == "fisher_exact":
            _, res.fisher_exact = fisher_exact(table, alternative=alt)
        
        else:
            setattr(res, fn.__name__, fn(table, alternative=alt).pvalue)
    return res


def count_treshold(map_obj, alpha):
    def cond(res: ResPvalues):
        """
        Condition if we accept H_0 or not
        """
        boschloo = 1 if res.boschloo_exact > alpha else 0
        barnard = 1 if res.barnard_exact > alpha else 0
        fisher = 1 if res.fisher_exact > alpha else 0
        return np.array([boschloo, barnard, fisher])
    
    return list(
        reduce(lambda x, y: x + y, map(cond, map_obj), np.array([0,0,0]))
    )
            

In [ ]:
def count_rejection_hypothesis(alpha=.05, repeat=1):
    def mapping_samples(n1, n2, sample_tuple):
        a, b = sample_tuple
        return get_pvalues([[a, b], [n1 - a, n2 - b]], alt="less")
    
    for _ in range(repeat):
        # p_1 = .2 < p_2 = .3. Therefore, H_0 is false
        n1, p1, n2, p2 = 33, .2, 62, .3
        size = 10
        left_side = get_binom_samples(n1, p1, size)
        right_side = get_binom_samples(n2, p2, size)
        mapping_fn = partial(mapping_samples, n1, n2)

        # Let's compute power = 1 - 𝛽. Since 𝛽 = P(type II error) = P_Ha(Accept H_0)    
        map_obj = list(map(mapping_fn, zip(left_side, right_side)))
        print(count_treshold(map_obj, alpha=alpha))
        return 

   
%time count_rejection_hypothesis()

Let's do this in parallele now

In [ ]:
from power_comparison_with_processus import count_rejection_hypothesis_with_workers
res = count_rejection_hypothesis_with_workers(repeat=6)